In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [2]:
! pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.7 MB/s eta 0:00:00


In [3]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=ba713d421f5a894d9b23072d550aba6180cd7d8eb52e0057acd1c4af5a605dd0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
# 데이터 처리 및 분석
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 분자 데이터 처리
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, Crippen, rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import GraphDescriptors
from rdkit.Chem import rdchem
from rdkit.Chem import EnumerateStereoisomers
from rdkit.Chem import ResonanceMolSupplier

# 머신러닝
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.manifold import TSNE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, lit, udf
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import IntegerType, FloatType, StringType, ArrayType

# 딥러닝
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D

# 기타 유틸리티
import os
import sys
import json
import warnings
from collections import defaultdict

# 경고 메시지 무시 설정
warnings.filterwarnings('ignore')

2024-04-18 14:21:21.101462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 14:21:21.101641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 14:21:21.289699: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# SparkSession 생성
spark = SparkSession.builder \
        .appName("Leash BELKA Test Data Load") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 14:21:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Parquet 파일 로드
train = spark.read.parquet("/kaggle/input/leash-BELKA/test.parquet")

In [7]:
# 분자 무게를 계산하는 UDF 생성
def calculate_mw(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return Descriptors.MolWt(molecule) if molecule else None

calculate_mw_udf = udf(calculate_mw, FloatType())

# 각 building block과 molecule의 분자 무게를 계산하여 새로운 열로 추가
train = train.withColumn("mw_buildingblock1", calculate_mw_udf(train["buildingblock1_smiles"]))
train = train.withColumn("mw_buildingblock2", calculate_mw_udf(train["buildingblock2_smiles"]))
train = train.withColumn("mw_buildingblock3", calculate_mw_udf(train["buildingblock3_smiles"]))
train = train.withColumn("mw_molecule", calculate_mw_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042|
|295246831| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|         HSA|          363.413|          119.167|          119.167|    615.042|
|295246832| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|         sEH

In [8]:
# 로그 P 값을 계산하는 UDF 생성
def calculate_logP(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return Crippen.MolLogP(molecule) if molecule else None

calculate_logP_udf = udf(calculate_logP, FloatType())

# molecule SMILES로부터 로그 P 값을 계산하여 새로운 열로 추가
train = train.withColumn("logP", calculate_logP_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8093|
|295246831| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|         HSA|          363.413|          119.167|          119.167|    615.042| 4.8093|
|295246832| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc

In [9]:
# 수소결합 기증자(HBD) 수를 계산하는 UDF 생성
def calculate_hbd(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return rdMolDescriptors.CalcNumHBD(molecule) if molecule else None

# 수소결합 수용체(HBA) 수를 계산하는 UDF 생성
def calculate_hba(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return rdMolDescriptors.CalcNumHBA(molecule) if molecule else None

# UDF 등록
calculate_hbd_udf = udf(calculate_hbd, IntegerType())
calculate_hba_udf = udf(calculate_hba, IntegerType())

# DataFrame에 수소결합 기증자와 수용체 수를 계산하여 새로운 열로 추가
train = train.withColumn("num_hbd", calculate_hbd_udf(train["molecule_smiles"]))
train = train.withColumn("num_hba", calculate_hba_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8093|      4|      7|
|295246831| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|         HSA|          363.413|          119.167|          119.167|    615.042| 4.8093|      4

In [10]:
# 회전 가능한 결합 수를 계산하는 UDF 생성
def calculate_rot_bonds(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return rdMolDescriptors.CalcNumRotatableBonds(molecule) if molecule else None

# UDF를 등록
calculate_rot_bonds_udf = udf(calculate_rot_bonds, IntegerType())

# DataFrame에 회전 가능한 결합 수를 계산하여 새로운 열로 추가
train = train.withColumn("num_rotatable_bonds", calculate_rot_bonds_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8093|      4|      7|                 12|
|295246831| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|         HSA| 

In [11]:
# 고리 수를 계산하는 UDF 생성
def calculate_ring_count(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return rdMolDescriptors.CalcNumRings(molecule) if molecule else None

# UDF를 등록
calculate_ring_count_udf = udf(calculate_ring_count, IntegerType())

# DataFrame에 고리 수를 계산하여 새로운 열로 추가
train = train.withColumn("num_rings", calculate_ring_count_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8093|      4|      7|                 12|        3|
|295246831| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N

In [12]:
# 부분 구조 검색 결과를 계산하는 UDF 생성
def calculate_substructure_count(smiles, substruct_smiles):
    molecule = Chem.MolFromSmiles(smiles)
    substructure = Chem.MolFromSmiles(substruct_smiles)
    return len(molecule.GetSubstructMatches(substructure)) if molecule and substructure else 0

# UDF 등록 (이번 경우에는 부분 구조 SMILES도 인자로 전달)
substruct_smiles = 'c1ccccc1'
calculate_substructure_count_udf = udf(lambda smiles: calculate_substructure_count(smiles, substruct_smiles), IntegerType())

# DataFrame에 부분 구조 개수를 계산하여 새로운 열로 추가
column_name = f'num_{substruct_smiles}'
train = train.withColumn(column_name, calculate_substructure_count_udf(col("molecule_smiles")))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8093|      4|      7|                 12|        3|           2|
|295246831| C#CCCC[C

In [13]:
# TPSA를 계산하는 UDF 생성
def calculate_tpsa(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return rdMolDescriptors.CalcTPSA(molecule) if molecule else 0

calculate_tpsa_udf = udf(calculate_tpsa, FloatType())

# DataFrame에 TPSA를 계산하여 새로운 열로 추가
train = train.withColumn("tpsa", calculate_tpsa_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8093|      4|      7|                 12|        3|           2|

In [14]:
# 분자 복잡성을 계산하는 UDF 생성
def calculate_complexity(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return GraphDescriptors.BertzCT(molecule) if molecule else 0

calculate_complexity_udf = udf(calculate_complexity, FloatType())

# DataFrame에 분자 복잡성을 계산하여 새로운 열로 추가
train = train.withColumn("molecular_complexity", calculate_complexity_udf(train["molecule_smiles"]))

# 데이터프레임의 업데이트된 구조 및 데이터 확인
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|          363.413|          119.167|          119.167|    615.042| 4.8

In [15]:
# Morgan 지문을 계산하는 UDF 생성
def calculate_fingerprint(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        fp = AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024)
        return list(fp)
    else:
        return [0]*1024  # 모든 비트를 0으로 설정한 배열 반환

calculate_fingerprint_udf = udf(calculate_fingerprint, ArrayType(IntegerType()))

# DataFrame에 화학적 지문을 계산하여 새로운 열로 추가
train = train.withColumn("chemical_fingerprint", calculate_fingerprint_udf(train["molecule_smiles"]))

train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|
+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+
|295246830| C#CCCC[C@H](NC(=O...|       C=Cc1ccc(N)cc1|       C=Cc1ccc(N)cc1|C#CCCC[C@H](Nc1nc...|        BRD4|       

In [16]:
# SMARTS 패턴을 사용하여 기능적 그룹 정의
functional_groups = {
    'alcohol': '[OH]',
    'carboxylic_acid': 'C(=O)[OH]',
    'amine': '[NX3;H2,H1,H0;!$(NC=O)]',
    'ketone': '[CX3](=O)[#6]',
    'ammonium': '[NX4+]',
    'ester': 'C(=O)O',
    'nitrile': 'C#N',
    'thiol': '[SH]',
    'sulfide': '[SX2]',
    'sulfoxide': 'S(=O)C',
    'aldehyde': 'C=O',
    'halogen': '[F,Cl,Br,I]'
}

# 각 SMILES에 대해 각 기능적 그룹의 수를 계산하는 UDF 생성
def count_groups(smiles, pattern):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        substructure = Chem.MolFromSmarts(pattern)
        return len(molecule.GetSubstructMatches(substructure))
    return 0

# 각 기능적 그룹에 대해 UDF 생성 및 데이터프레임에 열 추가
for name, pattern in functional_groups.items():
    count_groups_udf = udf(lambda x: count_groups(x, pattern), IntegerType())
    train = train.withColumn(f'count_{name}', count_groups_udf(col("molecule_smiles")))

train.show()

24/04/18 14:22:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|count_amine|count_ketone|count_ammonium|count_ester|count_nitrile|count_thiol|count_sulfide|count_sulfoxide|count_aldehyde|count_halogen|
+---------+---------------------+---------------------+-----------

In [17]:
# 루이스 산과 염기 패턴 정의
lewis_acids = {
    'boron_compounds': '[B]',  # 보론 화합물은 전형적인 루이스 산
    'phosphorus_compounds': '[PH2]',  # 포스포루스 화합물
}

lewis_bases = {
    'amines': '[N]',  # 아민
    'ethers': '[O]',  # 에테르
}

# 전체 패턴 딕셔너리 합치기
all_patterns = {**lewis_acids, **lewis_bases}

# 각 SMILES에 대해 해당 패턴의 수를 계산하는 UDF 생성
def count_substructures(smiles, pattern):
    mol = Chem.MolFromSmiles(smiles)
    substruct = Chem.MolFromSmarts(pattern)
    return len(mol.GetSubstructMatches(substruct)) if mol else 0

# 각 패턴에 대해 UDF 등록 및 데이터프레임에 적용
for name, pattern in all_patterns.items():
    count_substructures_udf = udf(lambda x: count_substructures(x, pattern), IntegerType())
    train = train.withColumn(f'count_{name}', count_substructures_udf(col("molecule_smiles")))
    
train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+---------------------+--------------------------+------------+------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|count_amine|count_ketone|count_ammonium|count_ester|count_nitrile|count_thiol|count_sulfide|count_sulfoxide|count_aldehyde|count_

In [18]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType
from rdkit import Chem

# Hammett constants defined as before
hammett_constants = {
    'NO2': ("[N+](=O)[O-]", 0.78),  # Nitro group
    'CN': ("[C]#[N]", 0.66),        # Cyano group
    'COOH': ("C(=O)[OH]", 0.44),    # Carboxylic acid
    'F': ("[F]", 0.34),             # Fluorine
    'Cl': ("[Cl]", 0.23),           # Chlorine
    'Br': ("[Br]", 0.23),           # Bromine
    'I': ("[I]", 0.28),             # Iodine
    'OH': ("[OH]", -0.37),          # Hydroxy group
    'OCH3': ("[O][CH3]", -0.27),    # Methoxy group
    'NH2': ("[NH2]", -0.66)         # Amino group
}

def calculate_substituent_effects(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if not molecule:  # Check if the molecule is valid
        return 0.0

    effect_sum = 0.0
    for key, (pattern, value) in hammett_constants.items():
        substruct = Chem.MolFromSmarts(pattern)
        if substruct is None:  # Ensure the SMARTS pattern is valid
            continue
        if molecule.HasSubstructMatch(substruct):
            effect_sum += value

    return effect_sum

calculate_substituent_effects_udf = udf(calculate_substituent_effects, FloatType())

# Applying the UDF to calculate the substituent effect and add it as a new column
train = train.withColumn("substituent_effect", calculate_substituent_effects_udf(col("molecule_smiles")))

# Show the updated DataFrame
train.show()


+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+---------------------+--------------------------+------------+------------+------------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|count_amine|count_ketone|count_ammonium|count_ester|count_nitrile|count_thiol|count_sulfide|count_sulfoxide|co

In [19]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from rdkit import Chem
from rdkit.Chem import EnumerateStereoisomers

# 스테레오이성질체의 수를 계산하는 UDF 생성
def calculate_stereoisomers(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        isomers = list(EnumerateStereoisomers.EnumerateStereoisomers(molecule))
        return len(isomers)
    return 0

# UDF 등록
calculate_stereoisomers_udf = udf(calculate_stereoisomers, IntegerType())

# DataFrame에 스테레오이성질체 수를 계산하여 새로운 열로 추가
train = train.withColumn("stereoisomers_count", calculate_stereoisomers_udf(col("molecule_smiles")))

# DataFrame을 출력하여 결과 확인
train.show()


+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+---------------------+--------------------------+------------+------------+------------------+-------------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|count_amine|count_ketone|count_ammonium|count_ester|count_nitrile|count_thiol|count_sulfid

In [20]:
# 전기음성도 참조 데이터 (Pauling Scale)
electronegativities = {
    'H': 2.20, 'B': 2.04, 'C': 2.55, 'N': 3.04,
    'O': 3.44, 'F': 3.98, 'Si': 1.90, 'P': 2.19,
    'S': 2.58, 'Cl': 3.16, 'Br': 2.96, 'I': 2.66
}

# 공유결합 극성을 계산하는 UDF 생성
def calculate_bond_polarity(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    polarities = []
    if molecule:
        for bond in molecule.GetBonds():
            begin_atom = bond.GetBeginAtom().GetSymbol()
            end_atom = bond.GetEndAtom().GetSymbol()
            if begin_atom in electronegativities and end_atom in electronegativities:
                polarity = abs(electronegativities[begin_atom] - electronegativities[end_atom])
                polarities.append(polarity)
    return sum(polarities) / len(polarities) if polarities else 0

calculate_bond_polarity_udf = udf(calculate_bond_polarity, FloatType())

# DataFrame에 공유결합 극성 변수를 계산하여 새로운 열로 추가
train = train.withColumn("polarity_molecule", calculate_bond_polarity_udf(col("molecule_smiles")))
train = train.withColumn("polarity_bb1", calculate_bond_polarity_udf(col("buildingblock1_smiles")))
train = train.withColumn("polarity_bb2", calculate_bond_polarity_udf(col("buildingblock2_smiles")))
train = train.withColumn("polarity_bb3", calculate_bond_polarity_udf(col("buildingblock3_smiles")))

train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+---------------------+--------------------------+------------+------------+------------------+-------------------+-----------------+------------+------------+------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|count_amine|count_ketone|count_am

In [21]:
# 키랄 센터 수를 계산하는 UDF
def count_chiral_centers(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        chiral_centers = [atom for atom in mol.GetAtoms() if atom.GetChiralTag() != Chem.rdchem.ChiralType.CHI_UNSPECIFIED]
        return len(chiral_centers)
    return 0

# 이중 결합의 기하학적 이성질체 수를 계산하는 UDF
def count_geometric_isomers(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        double_bonds = [bond for bond in mol.GetBonds() if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE]
        geom_isomers = sum(1 for bond in double_bonds if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE)
        return geom_isomers
    return 0

# UDF 등록
count_chiral_centers_udf = udf(count_chiral_centers, IntegerType())
count_geometric_isomers_udf = udf(count_geometric_isomers, IntegerType())

# DataFrame에 입체화학적 특성 변수를 계산하여 새로운 열로 추가
train = train.withColumn("chiral_centers", count_chiral_centers_udf(col("molecule_smiles")))
train = train.withColumn("geometric_isomers", count_geometric_isomers_udf(col("molecule_smiles")))

train.show()

+---------+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+---------------------+--------------------------+------------+------------+------------------+-------------------+-----------------+------------+------------+------------+--------------+-----------------+
|       id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|

In [22]:
train = train.drop('id')
train.show()

+---------------------+---------------------+---------------------+--------------------+------------+-----------------+-----------------+-----------------+-----------+-------+-------+-------+-------------------+---------+------------+------+--------------------+--------------------+-------------+---------------------+-----------+------------+--------------+-----------+-------------+-----------+-------------+---------------+--------------+-------------+---------------------+--------------------------+------------+------------+------------------+-------------------+-----------------+------------+------------+------------+--------------+-----------------+
|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|mw_buildingblock1|mw_buildingblock2|mw_buildingblock3|mw_molecule|   logP|num_hbd|num_hba|num_rotatable_bonds|num_rings|num_c1ccccc1|  tpsa|molecular_complexity|chemical_fingerprint|count_alcohol|count_carboxylic_acid|count_amine|count_ke

In [23]:
train

DataFrame[buildingblock1_smiles: string, buildingblock2_smiles: string, buildingblock3_smiles: string, molecule_smiles: string, protein_name: string, mw_buildingblock1: float, mw_buildingblock2: float, mw_buildingblock3: float, mw_molecule: float, logP: float, num_hbd: int, num_hba: int, num_rotatable_bonds: int, num_rings: int, num_c1ccccc1: int, tpsa: float, molecular_complexity: float, chemical_fingerprint: array<int>, count_alcohol: int, count_carboxylic_acid: int, count_amine: int, count_ketone: int, count_ammonium: int, count_ester: int, count_nitrile: int, count_thiol: int, count_sulfide: int, count_sulfoxide: int, count_aldehyde: int, count_halogen: int, count_boron_compounds: int, count_phosphorus_compounds: int, count_amines: int, count_ethers: int, substituent_effect: float, stereoisomers_count: int, polarity_molecule: float, polarity_bb1: float, polarity_bb2: float, polarity_bb3: float, chiral_centers: int, geometric_isomers: int]

In [24]:
train.write.parquet("/kaggle/working/test_fe_single.parquet", mode="overwrite")